In [1]:
import pickle 
import torch
import os
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM

from unlearning.metrics import calculate_metrics_rmu, get_loss_added_rmu_model
from unlearning.tool import get_hf_model, get_basic_gemma_2b_it_layer9_act_store
from transformer_lens import HookedTransformer

torch.set_grad_enabled(False)

In [3]:
base_model = HookedTransformer.from_pretrained('google/gemma-2-2b-it', dtype=torch.bfloat16)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer


In [4]:
from sae.activation_store import ActivationsStore
from unlearning.jump_relu import load_gemma2_2b_sae

sae = load_gemma2_2b_sae(layer=0)
sae.cfg.dataset = "Skylion007/openwebtext"
sae.cfg.n_batches_in_store_buffer = 8

activation_store = ActivationsStore(sae.cfg, base_model)

Found SAE with l0=105 at path google/gemma-scope-2b-pt-res/layer_0/width_16k/average_l0_105/params.npz


creating data loader
buffer
dataloader


In [5]:
dataset_names = ['wmdp-bio', 'high_school_us_history', 'high_school_geography', 'college_computer_science', 'human_aging', 'college_biology']
question_type = 'correct'
metric_params = {d: {'target_metric': question_type} for d in dataset_names}
hf_model_name = "google/gemma-2-2b-it"

for layer in [3, 7, 11]:
    for s in [100, 200, 400]:
        for a in [100, 300, 500]:
            rmu_model_name = f'yeutong/gemma_2_2b_it_s{s}_a{a}_layer{layer}'       
            
            half_rmu_model_name = rmu_model_name.split('/')[-1]
            half_hf_model_name = hf_model_name.split('/')[-1]
            file_name = f'../data/unlearn_results/{half_hf_model_name}/rmu/{question_type}/{half_rmu_model_name}.pkl'
            if os.path.exists(file_name):
                continue
            # use hf
            # rmu_model = get_hf_model(rmu_model_name, hf_model_name)
            
            # use transformer lens
            hf_model = AutoModelForCausalLM.from_pretrained(rmu_model_name)
            rmu_model = HookedTransformer.from_pretrained(hf_model_name, hf_model=hf_model)
            
            results = calculate_metrics_rmu(rmu_model, dataset_names, metric_params=metric_params)

            loss_return = get_loss_added_rmu_model(rmu_model, base_model, activation_store, n_batch=50)
            loss_added = loss_return[0].mean()

                
            results['loss_return'] = loss_return
            results['loss_added'] = loss_added
            print(rmu_model_name, loss_added)
            
            
            with open(file_name, 'wb') as f:
                pickle.dump(results, f)
                
            del rmu_model, hf_model
            torch.cuda.empty_cache()
            gc.collect()
            

config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

/root/unlearning/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 4988.03 MB. The target location /root/.cache/huggingface/hub/models--yeutong--gemma_2_2b_it_s400_a100_layer11/blobs only has 3744.53 MB free disk space.
  warnings.warn(


model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [14]:
# load pickle

with open(f'../data/unlearn_results/rmu/all/{model_name}.pkl', 'rb') as f:
    results = pickle.load(f)

In [15]:
for dataset in results:
    print(f'{dataset}: {results[dataset]["mean_correct"]}')

wmdp-bio: 0.34878242015838623
high_school_us_history: 0.7549020051956177
high_school_geography: 0.7828282713890076
college_computer_science: 0.44999998807907104
human_aging: 0.6233184337615967
college_biology: 0.7013888955116272


In [15]:
for dataset in results:
    print(f'{dataset}: {results[dataset]["mean_correct"]}')

wmdp-bio: 0.42337164282798767
high_school_us_history: 1.0
high_school_geography: 1.0
college_computer_science: 1.0
human_aging: 1.0
college_biology: 0.9863013625144958


In [16]:
dataset_names = ['wmdp-bio', 'high_school_us_history', 'high_school_geography', 'college_computer_science', 'human_aging', 'college_biology']
metric_params = {d: {'target_metric': 'all'} for d in dataset_names}
results_all = calculate_metrics_rmu(rmu_model, dataset_names, metric_params=metric_params)

100%|██████████| 24/24 [00:02<00:00, 11.43it/s]


In [17]:
for dataset in results_all:
    print(f'{dataset}: {results_all[dataset]["mean_correct"]}')

wmdp-bio: 0.34878242015838623
high_school_us_history: 0.7549020051956177
high_school_geography: 0.7828282713890076
college_computer_science: 0.44999998807907104
human_aging: 0.6233184337615967
college_biology: 0.7013888955116272


: 